# Azure Cognitive Search - Code Sample for chunking documents and generating vector embeddings via indexer composition

This code demonstrates a pattern of composing multiple indexers to ingest content from blob storage documents, chunk them, generate embeddings and store them as their own documents in a search index. The code sample here will demonstrate storing the chunked document fragments in their own index, but users can choose to co-locate them in the same index if needed. The following image describes this composition pattern

![Indexer chunk composition](../data/images/indexer-composition.png)

## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [1]:
! pip install azure-search-documents --pre
! pip install openai
! pip install openai[datalib]
! pip install python-dotenv
! pip install azure-storage-blob

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for azure-search-documents from https://files.pythonhosted.org/packages/25/f4/ec7c1d6bafb037d3017db93ef44e18efe84e6d4e7b8906153a9bb777786e/azure_search_documents-11.4.0b8-py3-none-any.whl.metadata
  Obtaining dependency information for azure-core<2.0.0,>=1.24.0 from https://files.pythonhosted.org/packages/98/3a/d53e2b8a75c448ef45d7ae4b0659eb6c0d48978f25a709e2a39894a48704/azure_core-1.29.4-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.5 MB/s eta 0:00:00
  Obtaining dependency information for requests>=2.18.4 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.6.0 from https://files.pythonhosted.org/packages/63/d6/ebc4ad51f7a42b0e46c42693563f38b4299314778a773a802a968d6ae742/typ

## Deploy the custom web API skill for chunking + embedding

This sample code also relies on a custom web api skill to be deployed to Azure functions. The custom web api skill performs chunking of content and then generates vector embeddings from the content utilizing Azure Open AI service. The code for the custom web api skill is available as an [Azure Cognitive Search power skill](https://github.com/Azure-Samples/azure-search-power-skills/blob/main/Vector/EmbeddingGenerator/README.md) and can be easily deployed via Visual Studio Code to Azure functions.

Please follow those steps first and ensure that the function app is running before proceeding with the rest of the sample.

## Configuring storage accounts for deletion detection

The storage accounts used for storing both the source documents as well as for storing the knowledge store projections (after the "chunking + embedding" step) need to adhere to the following requirements, in order to seamlessly track document deletes:

1. They need to be of type "Standard general-purpose v2".
2. They need to have soft delete enabled. Learn more [here](https://learn.microsoft.com/azure/storage/blobs/soft-delete-blob-enable?tabs=azure-portal)

Learn more about deletion detection policies used in Azure Cognitive Search [here](https://learn.microsoft.com/azure/search/search-howto-index-changed-deleted-blobs?tabs=portal#native-blob-soft-delete-preview)

## Manage the index, data source, skillset and indexer for the source document

The following code will configure an index to hold the source documents, via an indexer that reads data from an Azure storage container that is able to generate embeddings and write that to a separate storage account (knowledge store).

### Import required libraries

In [2]:
import os
import time
import requests
import openai
import os
import re
import logging
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceNotFoundError
from azure.search.documents.models import Vector  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchField,
    SearchableField,
    SearchFieldDataType,
    SearchIndexer,
    IndexingParameters,
    FieldMapping,
    FieldMappingFunction,
    InputFieldMappingEntry, 
    OutputFieldMappingEntry, 
    SearchIndexerSkillset,
    SearchIndexerKnowledgeStore,
    SearchIndexerKnowledgeStoreProjection,
    SearchIndexerKnowledgeStoreFileProjectionSelector,
    IndexingParameters, 
    WebApiSkill,
    SearchIndex,
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField,
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)

In [3]:
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_SEARCH_KNOWLEDGE_STORE_CONNECTION_STRING = os.getenv("AZURE_KNOWLEDGE_STORE_STORAGE_CONNECTION_STRING")

def get_index_client() -> SearchIndexClient:
    return SearchIndexClient(AZURE_SEARCH_SERVICE_ENDPOINT, AzureKeyCredential(AZURE_SEARCH_KEY))

def get_indexer_client() -> SearchIndexerClient:
    return SearchIndexerClient(AZURE_SEARCH_SERVICE_ENDPOINT, AzureKeyCredential(AZURE_SEARCH_KEY))

def get_index_name(index_prefix):
    return f"{index_prefix}-index"

def get_datasource_name(index_prefix):
    return f"{index_prefix}-datasource"

def get_skillset_name(index_prefix):
    return f"{index_prefix}-skillset"

def get_indexer_name(index_prefix):
    return f"{index_prefix}-indexer"

def get_chunk_index_blob_container_name(index_prefix):
    return f"{index_prefix}ChunkIndex".replace('-', '').lower()

def get_knowledge_store_connection_string():
    return AZURE_SEARCH_KNOWLEDGE_STORE_CONNECTION_STRING

### Define simple utilities to to help configure index, data source, skillset (with knowledge store) and indexer

In [4]:
def create_index(index_name, fields, vector_search, semantic_title_field_name, semantic_content_field_names):
    semantic_settings = SemanticSettings(
        configurations=[SemanticConfiguration(
            name='default',
            prioritized_fields=PrioritizedFields(
                title_field=SemanticField(field_name=semantic_title_field_name), prioritized_content_fields=[SemanticField(field_name=field_name) for field_name in semantic_content_field_names]))])
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
        semantic_settings=semantic_settings)
    index_client = get_index_client()
    return index_client.create_index(index)

In [5]:
def create_blob_datasource(datasource_name, storage_connection_string, container_name):
    # This example utilizes a REST request as the python SDK doesn't support the blob soft delete policy yet
    api_version = '2023-07-01-Preview'
    headers = {
        'Content-Type': 'application/json',
        'api-key': f'{AZURE_SEARCH_KEY}'
    }
    data_source = {
        "name": datasource_name,
        "type": "azureblob",
        "credentials": {"connectionString": storage_connection_string},
        "container": {"name": container_name},
        "dataDeletionDetectionPolicy": {"@odata.type": "#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy"}
    }

    url = '{}/datasources/{}?api-version={}'.format(AZURE_SEARCH_SERVICE_ENDPOINT, datasource_name, api_version)
    response = requests.put(url, json=data_source, headers=headers)

    ds_client = get_indexer_client()
    return ds_client.get_data_source_connection(datasource_name)

In [6]:
def wait_for_indexer_completion(indexer_name):
    indexer_client = get_indexer_client()
    # poll status and wait until indexer is complete
    status = f"Indexer {indexer_name} not started yet"
    while (indexer_client.get_indexer_status(indexer_name).last_result == None) or ((status := indexer_client.get_indexer_status(indexer_name).last_result.status) != "success"):
        print(f"Indexing status:{status}")

        # It's possible that the indexer may reach a state of transient failure, especially when generating embeddings
        # via Open AI. For the purposes of the demo, we'll just break out of the loop and continue with the rest of the steps.
        if (status == "transientFailure"):
            print(f"Indexer {indexer_name} failed before fully indexing documents")
            break
        time.sleep(5)

## Utilities to manage the "source" document index

In [7]:
class DocumentIndexManager():
    def _create_document_index(self, index_prefix):
        name = get_index_name(index_prefix)
        fields = [
            SimpleField(name="document_id", type=SearchFieldDataType.String, filterable=True, sortable=True, key=True),
            SearchableField(name="content", type=SearchFieldDataType.String),
            SimpleField(name="filesize", type=SearchFieldDataType.Int64),
            SimpleField(name="filepath", type=SearchFieldDataType.String)
        ]
        return create_index(name, fields, vector_search=None, semantic_title_field_name="filepath", semantic_content_field_names=["content"])

    def _create_document_datasource(self, index_prefix, storage_connection_string, container_name):
        name = get_datasource_name(index_prefix)
        return create_blob_datasource(name, storage_connection_string, container_name)

    def _create_document_skillset(self, index_prefix, content_field_name="content"):
        embedding_skill_endpoint = os.getenv("AZURE_SEARCH_EMBEDDING_SKILL_ENDPOINT")

        name = get_skillset_name(index_prefix)
        chunk_index_blob_container_name = get_chunk_index_blob_container_name(index_prefix)
        content_context = f"/document/{content_field_name}"
        embedding_skill = WebApiSkill(
                            name="chunking-embedding-skill",
                            uri=embedding_skill_endpoint,
                            timeout="PT3M",
                            batch_size=1,
                            degree_of_parallelism=1,
                            context=content_context,
                            inputs=[
                                    InputFieldMappingEntry(name="document_id", source="/document/document_id"),
                                    InputFieldMappingEntry(name="text", source=content_context),
                                    InputFieldMappingEntry(name="filepath", source="/document/filepath"),
                                    InputFieldMappingEntry(name="fieldname", source=f"='{content_field_name}'")],
                            outputs=[OutputFieldMappingEntry(name="chunks", target_name="chunks")])
        knowledge_store = SearchIndexerKnowledgeStore(storage_connection_string=get_knowledge_store_connection_string(),
                                                    projections=[
                                                                SearchIndexerKnowledgeStoreProjection(
                                                                    objects=[SearchIndexerKnowledgeStoreFileProjectionSelector(
                                                                        storage_container=chunk_index_blob_container_name,
                                                                        generated_key_name="id",
                                                                        source_context=f"{content_context}/chunks/*",
                                                                        inputs=[
                                                                            InputFieldMappingEntry(name="source_document_id", source="/document/document_id"),
                                                                            InputFieldMappingEntry(name="source_document_filepath", source="/document/filepath"),
                                                                            InputFieldMappingEntry(name="source_field_name", source=f"{content_context}/chunks/*/embedding_metadata/fieldname"),
                                                                            InputFieldMappingEntry(name="title", source=f"{content_context}/chunks/*/title"),
                                                                            InputFieldMappingEntry(name="text", source=f"{content_context}/chunks/*/content"),
                                                                            InputFieldMappingEntry(name="embedding", source=f"{content_context}/chunks/*/embedding_metadata/embedding"),
                                                                            InputFieldMappingEntry(name="index", source=f"{content_context}/chunks/*/embedding_metadata/index"),
                                                                            InputFieldMappingEntry(name="offset", source=f"{content_context}/chunks/*/embedding_metadata/offset"),
                                                                            InputFieldMappingEntry(name="length", source=f"{content_context}/chunks/*/embedding_metadata/length")                                                                            
                                                                            ]
                                                                            )
                                                                    ]),
                                                                SearchIndexerKnowledgeStoreProjection(
                                                                files=[SearchIndexerKnowledgeStoreFileProjectionSelector(
                                                                    storage_container=f"{chunk_index_blob_container_name}images",
                                                                    generated_key_name="imagepath",
                                                                    source="/document/normalized_images/*",
                                                                    inputs=[]
                                                                        )
                                                                ])
                                                                ])
        skillset = SearchIndexerSkillset(name=name, skills=[embedding_skill], description=name, knowledge_store=knowledge_store)
        client = get_indexer_client()
        return client.create_skillset(skillset)

    def _create_document_indexer(self, index_prefix, data_source_name, index_name, skillset_name, content_field_name="content", generate_page_images=True):
        content_context = f"/document/{content_field_name}"
        name = get_indexer_name(index_prefix)
        indexer_config = {"dataToExtract": "contentAndMetadata", "imageAction": "generateNormalizedImagePerPage"} if generate_page_images else {"dataToExtract": "contentAndMetadata"}
        parameters = IndexingParameters(max_failed_items = -1, configuration=indexer_config)
        indexer = SearchIndexer(
            name=name,
            data_source_name=data_source_name,
            target_index_name=index_name,
            skillset_name=skillset_name,
            field_mappings=[FieldMapping(source_field_name="metadata_storage_path", target_field_name="document_id", mapping_function=FieldMappingFunction(name="base64Encode", parameters=None)),
                            FieldMapping(source_field_name="metadata_storage_name", target_field_name="filepath"),
                            FieldMapping(source_field_name="metadata_storage_size", target_field_name="filesize")],
            output_field_mappings=[],
            parameters=parameters
        )
        indexer_client = get_indexer_client()
        return indexer_client.create_indexer(indexer)

    def create_document_index_resources(self, index_prefix, customer_storage_connection_string, customer_container_name) -> dict:
        index_name = self._create_document_index(index_prefix).name
        data_source_name = self._create_document_datasource(index_prefix, customer_storage_connection_string, customer_container_name).name
        skillset_name = self._create_document_skillset(index_prefix).name    
        time.sleep(5)
        indexer_name = self._create_document_indexer(index_prefix, data_source_name, index_name, skillset_name).name
        wait_for_indexer_completion(indexer_name)
        return {"index_name": index_name, "data_source_name": data_source_name, "skillset_name": skillset_name, "indexer_name": indexer_name}

    def delete_document_index_resources(self, index_prefix):
        index_client = get_index_client()
        indexer_client = get_indexer_client()

        index_client.delete_index(index=get_index_name(index_prefix))
        indexer_client.delete_indexer(indexer=get_indexer_name(index_prefix))
        indexer_client.delete_data_source_connection(data_source_connection=get_datasource_name(index_prefix))
        indexer_client.delete_skillset(skillset=get_skillset_name(index_prefix))

        # delete the knowledge store tables and blobs
        knowledge_store_connection_string  = get_knowledge_store_connection_string()
        
        # delete the container directly from storage
        try:
            blob_service = BlobServiceClient.from_connection_string(knowledge_store_connection_string)
            blob_service.delete_container(get_chunk_index_blob_container_name(index_prefix))
        # handle resource not found error
        except ResourceNotFoundError:
            pass

## Utilities to manage the "chunked" document index - with vector embeddings

In [8]:
class ChunkIndexManager():

    def _create_chunk_index(self, index_prefix):
        name = get_index_name(f"{index_prefix}-chunk")
        vector_search = VectorSearch(
            algorithm_configurations=[
                HnswVectorSearchAlgorithmConfiguration(
                    name="my-vector-config",
                    kind="hnsw",
                    parameters={
                        "m": 4,
                        "efConstruction": 400,
                        "efSearch": 1000,
                        "metric": "cosine"
                    }
                )
            ]
        )
        fields = [
            SimpleField(name="id", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True, key=True),            
            SimpleField(name="source_document_id", type=SearchFieldDataType.String),
            SimpleField(name="source_document_filepath", type=SearchFieldDataType.String),
            SimpleField(name="source_field_name", type=SearchFieldDataType.String),
            SearchableField(name="title", type=SearchFieldDataType.String),   
            SimpleField(name="index", type=SearchFieldDataType.Int64),
            SimpleField(name="offset", type=SearchFieldDataType.Int64),
            SimpleField(name="length", type=SearchFieldDataType.Int64),
            SimpleField(name="hash", type=SearchFieldDataType.String),
            SearchableField(name="text", type=SearchFieldDataType.String),                 
            SearchField(name="embedding", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config")    
        ]
        index = create_index(name, fields, vector_search=vector_search, semantic_title_field_name="title", semantic_content_field_names=["text"])
        return index
    
    def _create_chunk_datasource(self, index_prefix, storage_connection_string, container_name):
        name = get_datasource_name(f"{index_prefix}-chunk")
        return create_blob_datasource(name, storage_connection_string, container_name)

    def _create_chunk_indexer(self, index_prefix, data_source_name, index_name):
        name = get_indexer_name(f"{index_prefix}-chunk")
        parameters = IndexingParameters(configuration={"parsing_mode": "json"})
        indexer = SearchIndexer(
            name=name,
            data_source_name=data_source_name,
            target_index_name=index_name,
            parameters=parameters
        )
        indexer_client = get_indexer_client()
        return indexer_client.create_indexer(indexer)


    def create_chunk_index_resources(self, index_prefix) -> dict:
        chunk_index_storage_connection_string = get_knowledge_store_connection_string()
        chunk_index_blob_container_name = get_chunk_index_blob_container_name(index_prefix)

        index_name = self._create_chunk_index(index_prefix).name
        data_source_name = self._create_chunk_datasource(index_prefix, chunk_index_storage_connection_string, chunk_index_blob_container_name).name
        time.sleep(5)
        indexer_name = self._create_chunk_indexer(index_prefix, data_source_name, index_name).name
        wait_for_indexer_completion(indexer_name)
        return {"index_name": index_name, "data_source_name": data_source_name, "indexer_name": indexer_name}


    # delete all the resources
    def delete_chunk_index_resources(self, index_prefix):
        index_client = get_index_client()
        indexer_client = get_indexer_client()

        index_client.delete_index(index=f"{index_prefix}-chunk-index")
        indexer_client.delete_indexer(indexer=f"{index_prefix}-chunk-indexer")
        indexer_client.delete_data_source_connection(data_source_connection=f"{index_prefix}-chunk-datasource")


## Text embedder utility to aid during query time

**NOTE**: Make sure to utilize the same Azure OpenAI Embedding Deployment at query time as the one used in the custom web api skill.

In [9]:
class TextEmbedder():
    openai.api_type = "azure"    
    openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
    openai.api_base = f"https://{os.getenv('AZURE_OPENAI_SERVICE_NAME')}.openai.azure.com/"
    openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

    def clean_text(self, text, text_limit=7000):
        # Clean up text (e.g. line breaks, )    
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[\n\r]+', ' ', text).strip()
        # Truncate text if necessary (e.g. for, ada-002, 4095 tokens ~ 7000 chracters)    
        if len(text) > text_limit:
            logging.warning("Token limit reached exceeded maximum length, truncating...")
            text = text[:text_limit]
        return text

    # Function to generate embeddings for title and content fields, also used for query embeddings
    def generate_embeddings(self, text, clean_text=True):
        if clean_text:
            text = self.clean_text(text)
        response = openai.Embedding.create(input=text, engine=self.AZURE_OPENAI_EMBEDDING_DEPLOYMENT)
        embeddings = response['data'][0]['embedding']
        return embeddings

## Wire up the utilities

In [10]:
def create_indexes(prefix, customer_storage_connection_string, container_name):
    index_manager = DocumentIndexManager()
    doc_index_resources = index_manager.create_document_index_resources(prefix, customer_storage_connection_string, container_name)

    time.sleep(5)

    chunk_index_manager = ChunkIndexManager()
    chunk_index_resources = chunk_index_manager.create_chunk_index_resources(prefix)
    return {"doc_index_resources": doc_index_resources, "chunk_index_resources": chunk_index_resources}

def delete_indexes(prefix):
    index_manager = DocumentIndexManager()
    index_manager.delete_document_index_resources(prefix)
    chunk_index_manager = ChunkIndexManager()
    chunk_index_manager.delete_chunk_index_resources(prefix)


## Putting it all together

The following code will upload a bunch of sample PDFs to the "source document" storage account, in the container specified. And will implement the indexer composition pattern to ingest both the content from the source documents as well as the chunked + embedded content.

### Upload the sample data to blob storage

In [11]:
tenant ='pythonsample'

customer_storage_connection_string = os.getenv("DOCUMENT_AZURE_STORAGE_CONNECTION_STRING")
container_name = os.getenv("DOCUMENT_AZURE_STORAGE_CONTAINER_NAME")

prefix = f"{tenant}-{container_name}"

# Delete any existing Azure Cognitive Search resources
delete_indexes(prefix)

blob_service_client = BlobServiceClient.from_connection_string(customer_storage_connection_string)
container_client = blob_service_client.get_container_client(container=container_name)

if not container_client.exists():
    container_client.create_container()

# Upload sample documents to blob storage
for root, dirs, files in os.walk("./data/documents/"):
    for file in files:
        with open(os.path.join(root, file), "rb") as data:
            container_client.upload_blob(file, data, overwrite=True)

### Create the Azure Cognitive Search resources

**NOTE**: The following example creates the source document indexer and the chunk document indexer, but we wait for the first indexer to fully finish its run before creating the second - this is reasonable with very small amounts of data, but wouldn't scale well for larger data. In that scenario it would make more sense to create the indexers in parallel with a schedule and let them run on their own and converge eventually.

In [12]:
# ensure indexes
index_resources = create_indexes(prefix, customer_storage_connection_string, container_name)

Subtype value #Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy has no mapping, use base class DataDeletionDetectionPolicy.


Indexing status:Indexer pythonsample-documents-indexer not started yet
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress
Indexing status:inProgress


Subtype value #Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy has no mapping, use base class DataDeletionDetectionPolicy.


Indexing status:Indexer pythonsample-documents-chunk-indexer not started yet
Indexing status:Indexer pythonsample-documents-chunk-indexer not started yet
Indexing status:inProgress


### Query the "chunk" search index with different kinds of queries

In [13]:
def query_vector_index(index_name, query, vector_only=False):  
    embedder = TextEmbedder()  
    vector = Vector(value=embedder.generate_embeddings(query), k=3, fields="embedding")  
    search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name, AzureKeyCredential(AZURE_SEARCH_KEY))  
    if vector_only:  
        search_text = None  
    else:  
        search_text = query  
    results = search_client.search(search_text=search_text, vectors=[vector], top=3)  
    return results  

**Vector only query**

In [14]:
chunk_index_name = index_resources["chunk_index_resources"]["index_name"]  
results = query_vector_index(chunk_index_name, "hearing aid", vector_only=True)  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Content: {result['text']}")  
    print(f"Source Document: {result['source_document_filepath']}")  

Title: Benefit_Options.pdf
Content: ntoso Electronics:

Next Steps 
We hope that this information has been helpful in understanding the differences between Northwind 
Health Plus and Northwind Standard. We are confident that you will find the right plan for you and 
your family. Thank you for choosing Contoso Electronics! 

pageimage3.jpg
Source Document: Benefit_Options.pdf
Title: employee_handbook.pdf
Content: nd Protection 

 

Contoso Electronics is committed to protecting the security of your personal information. 

We have implemented physical, technical, and administrative measures to protect your data 

from unauthorized access, alteration, or disclosure. 

 

We use secure servers and encryption technology to protect data transmitted over the 

Internet. 

 

Access to Personal Information 

 

You have the right to access, review, and request a copy of your personal information that 

we have collected and stored. You may also request that we delete or correct any inaccurate 

**Hybrid query**

In [15]:
chunk_index_name = index_resources["chunk_index_resources"]["index_name"]
results = query_vector_index(chunk_index_name, "hearing aid")
for result in results:
    print(f"Title: {result['title']}")  
    print(f"Content: {result['text']}")  
    print(f"Source Document: {result['source_document_filepath']}")  

Title: Benefit_Options.pdf
Content: ntoso Electronics:

Next Steps 
We hope that this information has been helpful in understanding the differences between Northwind 
Health Plus and Northwind Standard. We are confident that you will find the right plan for you and 
your family. Thank you for choosing Contoso Electronics! 

pageimage3.jpg
Source Document: Benefit_Options.pdf
Title: employee_handbook.pdf
Content: nd Protection 

 

Contoso Electronics is committed to protecting the security of your personal information. 

We have implemented physical, technical, and administrative measures to protect your data 

from unauthorized access, alteration, or disclosure. 

 

We use secure servers and encryption technology to protect data transmitted over the 

Internet. 

 

Access to Personal Information 

 

You have the right to access, review, and request a copy of your personal information that 

we have collected and stored. You may also request that we delete or correct any inaccurate 